In [1]:
from rllab.sampler.utils import rollout
import joblib
import tensorflow as tf

In [6]:
speedup = 1
max_path_length = 1000
file = "../data/local/2-14-dev-linear-ocm--branch-fix-manual-qf/2-14-dev-linear-ocm--branch-fix-manual-qf_2017_02_14_16_55_59_0000--s-0/params.pkl"


In [20]:
sess_context.__exit__(None, None, None)

In [21]:
tf.reset_default_graph()
sess = tf.Session()
sess_context = sess.as_default()
sess_context.__enter__()
data = joblib.load(file)

In [22]:
if 'policy' in data:
    policy = data['policy']
else:
    qf = data['optimizable_qfunction']
    policy = qf.implicit_policy
qf = data['eq']
env = data['env']

In [42]:
try:
    path = rollout(env, policy, max_path_length=max_path_length,
                   animated=False, speedup=speedup)
# Hack for now. Not sure why rollout assumes that close is an
# keyword argument
except TypeError as e:
    if (str(e) != "render() got an unexpected keyword "
                  "argument 'close'"):
        raise e

In [33]:
def to_onehot(x, num_values):
    onehot = np.zeros((1, num_values))
    onehot[0, x] = 1
    return onehot

In [44]:
env_action = np.array([[1, 0, 0]])
write_action = to_onehot(1, 3)
env_obs = to_onehot(1, 3)
memory = np.array([[0, 0, 5]])

action = env_action, write_action
obs = env_obs, memory
sess.run(
    qf.output,
    {
        qf.action_input: action,
        qf.observation_input: obs,
    }
)

array([[ 0.93430549]], dtype=float32)

In [43]:
path

{'actions': array([[ 0.,  1.,  0.,  0.,  1.,  0.]], dtype=float32),
 'agent_infos': {},
 'env_infos': {'target': array([2])},
 'observations': array([[ 0.,  1.,  0.,  0.,  0.,  0.]]),
 'rewards': array([1]),
 'terminals': array([ True], dtype=bool)}